In [114]:
#solution 1-c
#/For each iteration of the algorithm, new clusters are formed based on the old clusters. The iteration converges when
#the new step has no change in the clusters and remains same as the old step. If the new clustering is different 
#from the old then the newer one has a lower cost which means it is going towards convergence.
#Since the algorithm iterates a function whose domain is a finite set, the iteration must eventually enter a cycle. 
#The cycle cannot have length greater than 1 because otherwise there would be some clustering which has a 
#lower cost than itself which is impossible.
#Hence the cycle must have length exactly 1 and k-means converges in a finite number of iterations.

#since we are keeping the variables in each step i.e we fix the centroids and calculate the new mean of the cluster 
#and fixing new mean we calculate new centroids,this convergence goes towards local procedure minimum which does not
#guarantee optimality. Hence it does not converge to global minimum value.The value depends on the assumptions we fix
#in the beginning on the initial centroids

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator
from collections import Counter,OrderedDict
from scipy.sparse import csr_matrix,vstack
from scipy.sparse import find
import math
import time

In [115]:
def centroids_allclose(X, Y, atol = 1e-8):

    # If you want to check matrix shapes as well
    if np.array_equal(X.shape, Y.shape)==0:
        return False

    x1,y1,v1 = find(X)
    x2,y2,v2 = find(Y)
    index_match = np.array_equal(x1,x2) & np.array_equal(y1,y2)

    if index_match==0:
        return False
    else:  
        return np.allclose(v1,v2, atol=atol)

In [5]:
### Kmeans algorithm

In [6]:
def get_random_centroids(data,k):
    centroids=[]
    for i in range(0, k):
        centroids.append(data[np.random.randint(0, data.shape[0], size=1)])
    return centroids


In [7]:
def get_purity(clusters_label,N):
    purity=0.0
    for each_centroid in clusters_label:
        cluster_targets = clusters_label[each_centroid]
        frequency=Counter(cluster_targets)
        key_max=max(cluster_targets, key=cluster_targets.count)
        purity += frequency[key_max]
    purity = purity/N
    return purity

In [8]:
### sum(m[i][j]) where 0<=i<=number of true clusters
def get_Mj(m,j):
    Mj=0.0
    for i in range(m.shape[0]):
        Mj += m[i][j]
    return Mj

In [9]:
def column_gini_index(m,j):
    Gj=0.0
    Mj = get_Mj(m,j)
    for i in range(m.shape[0]):
         Gj+=math.pow(m[i][j]/Mj,2)
    return 1-Gj
            
            

In [10]:
def get_gini_index(m):
    gini_index = 0.0
    gini_index_denominator = 0.0
    gini_index_numerator = 0.0
    for j in range(m.shape[1]):
        Mj = get_Mj(m,j)
        gini_index_denominator += Mj
        gini_index_numerator += column_gini_index(m,j)*Mj
    gini_index = gini_index_numerator/gini_index_denominator
    return gini_index
        

In [81]:
def has_converged(old_centroids,centroids,iterations,iteration):
 
    if iteration>iterations:
        return True
    return np.allclose(old_centroids,centroids,atol=1e-2)
 
def KMeans(data,k,max_iterations,targets,actual_labels):
    #initial step: To choose random k centroids from the datapoints
    centroids = get_random_centroids(data,k)
    #targets = mnist.target
    prev_centroids = []
    converged = False
    iteration = 0 
    N = data.shape[0]

    while(not converged):
    
        distances = cosine_similarity(data,np.vstack(centroids))
        clusters = {}
        clusters_label = {}
        
        ##find closest centroids for each data point
        m = np.zeros((actual_labels, k))
        for data_point_index in range(0,N):
            centroid_max_index = np.argmax(distances[data_point_index])
            clusters.setdefault(centroid_max_index,[]).append(data[data_point_index])
            clusters_label.setdefault(centroid_max_index,[]).append(targets[data_point_index])
            true_cluster_index = int(targets[data_point_index])
            determined_cluster_index = centroid_max_index
            m[true_cluster_index][determined_cluster_index] += 1.0
        #print("iteration:"+str(iteration)+"..done making clusters")  
        prev_centroids = centroids
        centroids = [[] for i in range(k)]
        for each_cluster_index in clusters:
            centroids[each_cluster_index] = np.mean(np.vstack(clusters[each_cluster_index]),axis=0)
            
            #print("making centroids for cluster index:"+str(each_cluster_index))
        iteration+=1
        converged = has_converged(prev_centroids,centroids,max_iterations,iteration)
        #print("converged?:"+str(converged))
    
    purity = 0.0
    purity = get_purity(clusters_label,N)
    gini_index = 0.0
    gini_index = get_gini_index(m)
    print("#iteration:",iteration)
    print("purity:",purity)
    print("gini_index:",gini_index)
    

In [82]:
#load MNIST data
mnist = fetch_mldata('MNIST original')
normalized_dataset = np.divide(mnist.data,255)
mnist_data = normalized_dataset
mnist_target_data = mnist.target

In [91]:
print("MNIST KMeans")
start = time.time()
KMeans(mnist_data,10,55,mnist_target_data,10)
end = time.time()
print("Time taken:"+str(end - start)+" seconds")

MNIST KMeans
#iteration: 16
purity: 0.5991428571428571
gini_index: 0.5216824277782354
Time taken:39.0547821521759 seconds


In [116]:
#load 20NG data
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
twentyng_data = fetch_20newsgroups(subset='all',remove=('headers','footers','quotes'))
vectorizer = TfidfVectorizer(stop_words='english',max_df=0.95,min_df=2)
twentyng_data_vectors = vectorizer.fit_transform(twentyng_data.data)
twentyng_target = twentyng_data.target

In [117]:
def has_converged_20NG(old_centroids,centroids,iterations,iteration):
    if iteration>iterations:
        return True
    #print(centroids.shape)
    #print(old_centroids.shape)
    converged = centroids_allclose(old_centroids,centroids)
    return converged

def get_random_centroids_20NG(data,k):
    centroids = []
    for i in range(0, k):
        centroids.append(data[np.random.randint(0, data.shape[0], size=1)])
    return centroids 
    
def KMeans_20NG(data,k,max_iterations,targets):
    #initial step: To choose random k centroids from the datapoints
    centroids = get_random_centroids_20NG(data,k)
    converged = False
    iteration = 0 
    N = data.shape[0]
    prev_centroids = []
    actual_labels = 20
    
    while(not converged):
    
        distances = cosine_similarity(data,vstack(centroids))
        #print(type(distances))
        clusters = OrderedDict()
        clusters_label = OrderedDict()
        
        ##find closest centroids for each data point
        m = np.zeros((actual_labels, k))
        for data_point_index in range(0,N):
            centroid_max_index = np.argmax(distances[data_point_index])
            clusters.setdefault(centroid_max_index,[]).append(data[data_point_index])
            clusters_label.setdefault(centroid_max_index,[]).append(targets[data_point_index])
            true_cluster_index = int(targets[data_point_index])
            determined_cluster_index = centroid_max_index
            m[true_cluster_index][determined_cluster_index] += 1.0
        print("iteration:"+str(iteration)+"..done making clusters")  
        
        prev_centroids = centroids
        centroids = [[] for x in range(0,k)]
        for cluster_index in clusters:
            vstacked_datapoints = vstack(clusters[cluster_index])
            
            mean = vstacked_datapoints.mean(axis=0)
            sparsed_mean = csr_matrix(mean)
            centroids[cluster_index]=sparsed_mean
            #print("making centroids for cluster index:"+str(cluster_index))
        iteration+=1
        p = vstack(prev_centroids)
        print("centroids shape:",len(centroids))
        #print("centroids",centroids)
        c = vstack(centroids)
        converged = has_converged_20NG(p,c,max_iterations,iteration)
        #print("converged?:"+str(converged))    

    purity = 0
    purity = get_purity(clusters_label,N)
    gini_index = 0.0
    gini_index = get_gini_index(m)
    print("#iteration:",iteration)
    print("purity",purity)
    print("gini index",gini_index)
    

In [118]:
print("20NG Kmeans")
start = time.time()
KMeans_20NG(twentyng_data_vectors,20,100,twentyng_target)
end = time.time()
print("Time taken:"+str(end - start)+" seconds")

20NG Kmeans
iteration:0..done making clusters
centroids shape: 20
iteration:1..done making clusters
centroids shape: 20
iteration:2..done making clusters
centroids shape: 20
iteration:3..done making clusters
centroids shape: 20
iteration:4..done making clusters
centroids shape: 20
iteration:5..done making clusters
centroids shape: 20
iteration:6..done making clusters
centroids shape: 20
iteration:7..done making clusters
centroids shape: 20
iteration:8..done making clusters
centroids shape: 20
iteration:9..done making clusters
centroids shape: 20
iteration:10..done making clusters
centroids shape: 20
iteration:11..done making clusters
centroids shape: 20
iteration:12..done making clusters
centroids shape: 20
iteration:13..done making clusters
centroids shape: 20
iteration:14..done making clusters
centroids shape: 20
iteration:15..done making clusters
centroids shape: 20
iteration:16..done making clusters
centroids shape: 20
iteration:17..done making clusters
centroids shape: 20
iteratio

In [ ]:
# loading Fashion MNIST

In [119]:
import pandas as pd
fashion_data = pd.read_csv('fashion-mnist_train.csv')
targets_fashion = fashion_data.label.values
images = fashion_data.iloc[:,1:].values
images = images.astype(np.float)
print(images.shape)
normalized_images = np.divide(images,255)

(60000, 784)


In [120]:
print("Fashion MNIST KMeans")
start = time.time()
KMeans(normalized_images,10,100,targets_fashion,len(set(targets_fashion)))
end = time.time()
print("Time taken:"+str(end - start)+" seconds")

Fashion MNIST KMeans
#iteration: 43
purity: 0.6146
gini_index: 0.4923065706344012
Time taken:79.91207909584045 seconds
